# 测试包含插件的浏览器会话

本笔记本演示了如何使用PlaywrightSessionPool获取和使用包含插件的浏览器会话。

In [1]:
import sys, asyncio, os

if sys.platform == "win32":
    # 设置 ProactorEventLoop 策略
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

sys.stderr.fileno = lambda: 2
sys.path.append(os.path.join(os.getcwd(), '..'))

In [2]:
import random
from app.models.RPA_browser.browser_info_model import UserBrowserInfoListParams, UserBrowserInfoCreateParams
from app.utils.depends.session_manager import DatabaseSessionManager
import sys
import asyncio

from app.services.RPA_browser.browser_db_service import BrowserDBService

if sys.platform == 'win32':
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

from app.services.RPA_browser.browser_session_pool.playwright_pool import get_default_session_pool
import uuid

2025-12-09 15:00:51.772365: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [3]:
# 创建或获取浏览器指纹
browser_token = uuid.UUID("0fc69198bccb4b00928b9372c99190b7")

async with DatabaseSessionManager.async_session() as session:
    browser_list = await BrowserDBService.list_fingerprint(UserBrowserInfoListParams(
        browser_token=browser_token,
        page=1,
        per_page=10
    ), session)

if browser_list.items:
    browser_id = browser_list.items[0].id
else:
    async with DatabaseSessionManager.async_session() as session:
        created_fingerprint = await BrowserDBService.create_fingerprint(UserBrowserInfoCreateParams(
            browser_token=browser_token,
            fingerprint_int=random.randint(0, 9999)
        ), session)
        browser_id = created_fingerprint.id

print(f"Using browser_token: {browser_token}")
print(f"Using browser_id: {browser_id}")

Using browser_token: 0fc69198-bccb-4b00-928b-9372c99190b7
Using browser_id: 7403833021427617792


In [4]:
# 使用PlaywrightSessionPool获取包含插件的浏览器会话
from app.models.RPA_browser.browser_session_model import SessionCreateParams

# 获取会话池
session_pool = get_default_session_pool()

# 创建会话参数
session_params = SessionCreateParams(
    browser_token=browser_token,
    browser_id=browser_id,
    headless=True  # 使用有头模式方便观察
)

# 获取包含插件的会话
plugined_session = await session_pool.get_session(session_params)
print(f"Got plugined session for browser_id: {browser_id}")

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /flavour/CLIP-ViT-B-16-DataComp.XL-s13B-b90K/resolve/main/config.json (Caused by ConnectTimeoutError(<HTTPSConnection(host='huggingface.co', port=443) at 0x7dba644497f0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 462f4f49-fc34-4000-af8e-02ecedf7f4b2)')' thrown while requesting HEAD https://huggingface.co/flavour/CLIP-ViT-B-16-DataComp.XL-s13B-b90K/resolve/main/config.json


2025-12-09 15:01:14,241 WARNING '(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /flavour/CLIP-ViT-B-16-DataComp.XL-s13B-b90K/resolve/main/config.json (Caused by ConnectTimeoutError(<HTTPSConnection(host='huggingface.co', port=443) at 0x7dba644497f0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 462f4f49-fc34-4000-af8e-02ecedf7f4b2)')' thrown while requesting HEAD https://huggingface.co/flavour/CLIP-ViT-B-16-DataComp.XL-s13B-b90K/resolve/main/config.json


Retrying in 1s [Retry 1/5].


2025-12-09 15:01:14,243 WARNING Retrying in 1s [Retry 1/5].


'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /CIDAS/clipseg-rd64-refined/resolve/main/config.json (Caused by ConnectTimeoutError(<HTTPSConnection(host='huggingface.co', port=443) at 0x7dba64464410>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 3cbe0b01-e0fd-453a-a05b-1c95ffab2114)')' thrown while requesting HEAD https://huggingface.co/CIDAS/clipseg-rd64-refined/resolve/main/config.json


...

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /CIDAS/clipseg-rd64-refined/resolve/main/preprocessor_config.json (Caused by ConnectTimeoutError(<HTTPSConnection(host='huggingface.co', port=443) at 0x7dba64466990>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: bce6e39f-1974-4498-9e3f-e5d3b99f27e1)')' thrown while requesting HEAD https://huggingface.co/CIDAS/clipseg-rd64-refined/resolve/main/preprocessor_config.json


2025-12-09 15:01:14,311 WARNING '(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /CIDAS/clipseg-rd64-refined/resolve/main/preprocessor_config.json (Caused by ConnectTimeoutError(<HTTPSConnection(host='huggingface.co', port=443) at 0x7dba64466990>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: bce6e39f-1974-4498-9e3f-e5d3b99f27e1)')' thrown while requesting HEAD https://huggingface.co/CIDAS/clipseg-rd64-refined/resolve/main/preprocessor_config.json


Retrying in 1s [Retry 1/5].


2025-12-09 15:01:14,313 WARNING Retrying in 1s [Retry 1/5].


TargetClosedError: BrowserType.launch_persistent_context: Target page, context or browser has been closed
Browser logs:
Chromium sandboxing failed!
================================
To avoid the sandboxing issue, do either of the following:
  - (preferred): Configure your environment to support sandboxing
  - (alternative): Launch Chromium without sandbox using 'chromiumSandbox: false' option
================================

Call log:
  - <launching> /home/minato_aqua/bili-fastapi-browser-rpa/app/chrome/ungoogled-chromium-139.0.7258.154-1-x86_64.AppImage --disable-field-trial-config --disable-background-networking --disable-background-timer-throttling --disable-backgrounding-occluded-windows --disable-back-forward-cache --disable-breakpad --disable-client-side-phishing-detection --disable-component-extensions-with-background-pages --disable-component-update --no-default-browser-check --disable-default-apps --disable-dev-shm-usage --disable-extensions --disable-features=AcceptCHFrame,Av

In [ ]:
page = await plugined_session.get_current_page()

下面可以写需要测试的脚本了，写完了丢到自定义的类里就行

In [ ]:
await page.goto("https://live.bilibili.com/")
await page.click(".header-login-entry")
await page.fill(".bili-mini-account input", 'xxx')
await page.fill('.bili-mini-password input', 'xxx')

In [ ]:
await page.click(".universal-btn.login-btn")

In [ ]:
from app.services.geetest.captcha_break import acb

geetest_url = await page.locator(".geetest_item_wrap:last-of-type").evaluate(r"""el=>{
                                const style = window.getComputedStyle(el);
                                const bg = style.backgroundImage;
                                const match = bg.match(/url\(["']?(.*?)["']?\)/);
                                return match ? match[1] : null;
                                }""")
print(geetest_url)
result = await acb.predict_chinese_click_from_url(url=geetest_url)

In [ ]:
bound = await page.locator(".geetest_item_wrap:last-of-type").bounding_box()
for x, y in result:
    print(x, y)
    print(bound)
    click_position = {"x": x / 360 * bound.get('width'), "y": y / 360 * bound.get('height')}
    print(click_position)
    await asyncio.sleep(1)
    await page.locator(".geetest_table_box").click(position=click_position)
await page.screenshot(path='./geetest.png')


In [ ]:
await page.locator(".geetest_commit_tip").click()

In [ ]:
await page.screenshot(path='./login.png')

In [ ]:
# await page.goto("https://bot-detector.rebrowser.net/")
# await page.screenshot(path='./1.png',full_page=True)